## Model Evaluation - Balanced

In [ ]:
import numpy as np
import pandas as pd

from tensorflow.keras.models import load_model

from sklearn.metrics import accuracy_score, roc_auc_score, RocCurveDisplay, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

import pickle

Load the test sets

In [ ]:
# logistic regression and neural network
test_feats_file = "balanced_test_set.csv"
test_labels_file = "yale_new_haven_balanced_test_labels.csv"

In [ ]:
X_test = pd.read_csv(test_feats_file)
y_test = pd.read_csv(test_labels_file)

Get features

In [ ]:
test_ids = X_test['ID'].astype("int32")

In [ ]:
X_test = X_test[[col for col in X_test.columns if col != 'ID']]

In [ ]:
cc_cols = [col for col in X_test.columns if "cc_" in col]
pmh_cols = [col for col in X_test.columns if "pmh_" in col]
other_cols = [col for col in X_test.columns if col not in cc_cols and col not in pmh_cols]

In [ ]:
X_test = {'other': X_test[other_cols], 'cc': X_test[cc_cols], 'pmh': X_test[pmh_cols]}

Load the models

In [ ]:
nn_filepath = "pretrained_embedding_reduced"

In [ ]:
nn_model = load_model(nn_filepath)

### Test set probability

In [ ]:
nn_loss, nn_acc = nn_model.evaluate(X_test, y_test)

In [ ]:
print(f"Test Accuracy:\n\tNeural Network: {np.round(nn_acc*100, 2)}")

## AUROC

### Scores

In [ ]:
# Hong: AUROC = 0.91
nn_auroc = roc_auc_score(y_test, nn_model.predict(X_test))

In [ ]:
print(f"Test AUROC:\n\tNeural Network: {np.round(nn_auroc*100, 2)}")

### Plots

In [ ]:
RocCurveDisplay.from_predictions(y_test, nn_model.predict(X_test))
plt.show()

### Seniors vs. Adults

In [ ]:
senior_idxs_filename = "age_65_and_over.pickle"
adult_idxs_filename = "age_less_than_65.pickle"

In [ ]:
# indices of patients 65 and older
with open(senior_idxs_filename, 'rb') as f:
    senior_idxs = pickle.load(f)
    
with open(adult_idxs_filename, 'rb') as f:
    adult_idxs = pickle.load(f)

In [ ]:
seniors_test = test_ids[test_ids.isin(senior_idxs)].index
adults_test = test_ids[~test_ids.isin(senior_idxs)].index

Seniors vs. Adults Scores

In [ ]:
nn_loss_seniors, nn_acc_seniors = nn_model.evaluate({'other': X_test['other'].loc[seniors_test], 'cc': X_test['cc'].loc[seniors_test], 'pmh': X_test['pmh'].loc[seniors_test]}, y_test.loc[seniors_test])
nn_loss_adults, nn_acc_adults = nn_model.evaluate({'other': X_test['other'].loc[adults_test], 'cc': X_test['cc'].loc[adults_test], 'pmh': X_test['pmh'].loc[adults_test]}, y_test.loc[adults_test])

In [ ]:
# print(f"Training set:\n\tAdults score: {np.round(adult_score_train*100, 2)}\n\tSeniors score: {np.round(senior_score_train*100, 2)}")
print("Test Accuracy")
print()
print(f"Neural Network:\n\tAdults score: {np.round(nn_acc_adults*100, 2)} (n = {len(adults_test)})\n\tSeniors score: {np.round(nn_acc_seniors*100, 2)} (n = {len(seniors_test)})")

### Confusion Matrix

In [ ]:
y_pred_nn = np.round(nn_model.predict(X_test))

y_pred_nn_seniors = np.round(nn_model.predict({key: X_test[key].loc[seniors_test] for key in X_test.keys()}))
y_pred_nn_adults = np.round(nn_model.predict({key: X_test[key].loc[adults_test] for key in X_test.keys()}))

Overall

In [ ]:
norm = 'true'

print("Overall")
print('\tNeural Network')
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_nn, normalize=norm)

Seniors

In [ ]:
print('Seniors')
print('\tNeural Network')
ConfusionMatrixDisplay.from_predictions(y_test.loc[seniors_test], y_pred_nn_seniors, normalize=norm)

Adults

In [ ]:
print("Adults")
print('\tNeural Network')
ConfusionMatrixDisplay.from_predictions(y_test.loc[adults_test], y_pred_nn_adults, normalize=norm)